# BigQuery INFORMATION_SCHEMA の概要

`INFORMATION_SCHEMA` は、次に挙げるデータへのアクセスを提供する一連のビューです。<br>
このノートブックでは以下のメタデータの使い方を確認します


- [データセットのメタデータ](#データセットのメタデータ)
- [ジョブのメタデータ](#ジョブのメタデータ)
- [テーブルのメタデータ](#テーブルのメタデータ)


---
## データセットのメタデータ

データセットに関するメタデータには以下の二種類のものが存在します。
- SCHEMATA
- SCHEMATA_OPTIONS（ここでは割愛）

### SCHEMATA

`INFORMATION_SCHEMA.SCHEMATA` ビューにクエリを実行すると、現在のユーザーがアクセスできるプロジェクトの各データセットが 1 行で返されます。

`INFORMATION_SCHEMA.SCHEMATA` ビューのスキーマは次のとおりです。


|列名|データ型|値|
|---|---|---|
|CATALOG_NAME|STRING|データセットを含むプロジェクトの名前|
|SCHEMA_NAME|STRING	データセットの名前（datasetId ともいいます）|
|SCHEMA_OWNER|STRING|値は常に NULL|
|CREATION_TIME|TIMESTAMP|データセットの作成日時|
|LAST_MODIFIED_TIME|TIMESTAMP|データセットの最終更新日時|
|LOCATION|STRING|データセットの地理的なロケーション|

#### 構文

```sql
SELECT * FROM INFORMATION_SCHEMA.SCHEMATA
```

In [ ]:
%%bigquery ds
SELECT
 * EXCEPT(schema_owner)
FROM
 INFORMATION_SCHEMA.SCHEMATA

In [ ]:
ds.head()

---

## テーブルのメタデータ

`INFORMATION_SCHEMA` には、テーブル メタデータ用の以下のビューが含まれています。

- テーブルに関するメタデータ用の `TABLES` および `TABLE_OPTIONS`
- 列とフィールドに関するメタデータ用の `COLUMNS` および `COLUMN_FIELD_PATHS`

### TABLES ビュー

`INFORMATION_SCHEMA.TABLES` ビューにクエリを実行すると、クエリの結果として、データセット内の各テーブルまたはビューが 1 行で返されます。

`INFORMATION_SCHEMA.TABLES` ビューのスキーマは次のとおりです。

|列名|データ型|値|
|---|---|---|
|列名|データ型|値|
|TABLE_CATALOG|STRING|データセットを含むプロジェクトの名前|
|TABLE_SCHEMA|STRING|テーブルまたはビューを含むデータセットの名前（datasetId とも呼ばれる）|
|TABLE_NAME|STRING|テーブルまたはビューの名前（tableId とも呼ばれる）|
|TABLE_TYPE|STRING|テーブルタイプ:<br>BASE TABLE: BigQuery のテーブル<br>VIEW: BigQuery のビュー<br>EXTERNAL: 外部データソースを参照するテーブル|
|IS_INSERTABLE_INTO|STRING|YES または NO（テーブルが DML INSERT ステートメントに対応しているかどうかによる）|
|IS_TYPED|STRING|値は常に NO|
|CREATION_TIME|TIMESTAMP|テーブルの作成時間|

#### 構文

```sql
SELECT * FROM <Project-name>.<Dataset-name>.INFORMATION_SCHEMA.TABLES;
```

In [ ]:
%%bigquery
SELECT
 * EXCEPT(is_typed)
FROM
 `bigquery-public-data`.census_bureau_usa.INFORMATION_SCHEMA.TABLES

### COLUMNS ビュー

`INFORMATION_SCHEMA.COLUMNS` ビューにクエリを実行すると、クエリ結果として、テーブル内の列（フィールド）ごとに 1 行が表示されます。

`INFORMATION_SCHEMA.COLUMNS` ビューのスキーマは次のとおりです。

|列名|データ型|値|
|---|---|---|
|TABLE_CATALOG|STRING|データセットを含むプロジェクトの名前|
|TABLE_SCHEMA|STRING|テーブルを含むデータセットの名前（datasetId とも呼ばれる）|
|TABLE_NAME|STRING|テーブルまたはビューの名前（tableId とも呼ばれる）|
|COLUMN_NAME|STRING|列の名前|
|ORDINAL_POSITION|INT64|テーブル内の列の 1 から始まるオフセット。_PARTITIONTIME や _PARTITIONDATE などの疑似列の場合、値は NULL|
|IS_NULLABLE|STRING|YES または NO（列のモードが NULL 値を許可するかどうかによる）|
|DATA_TYPE|STRING|列の標準 SQL データ型|
|IS_GENERATED|STRING|値は常に NEVER|
|GENERATION_EXPRESSION|STRING|値は常に NULL|
|IS_STORED|STRING|値は常に NULL|
|IS_HIDDEN|STRING|YES または NO（列が _PARTITIONTIME や _PARTITIONDATE などの疑似列であるかどうかによる）|
|IS_UPDATABLE|STRING|値は常に NULL|
|IS_SYSTEM_DEFINED|STRING|YES または NO（列が _PARTITIONTIME や _PARTITIONDATE などの疑似列であるかどうかによる）|
|IS_PARTITIONING_COLUMN|STRING|YES または NO（列がパーティショニング列かどうかによる）|
|CLUSTERING_ORDINAL_POSITION|INT64|テーブルのクラスタリング列内の列の 1 から始まるオフセット。テーブルがクラスタ化テーブルでない場合、値は NULL|

#### 構文

```sql
SELECT
  * 
FROM <Project-name>.<Dataset-name>.INFORMATION_SCHEMA.COLUMNS;
WHERE table_name="<table_name>"
```



In [ ]:
%%bigquery
SELECT
 * EXCEPT(is_generated, generation_expression, is_stored, is_updatable)
FROM
 `bigquery-public-data`.wikipedia.INFORMATION_SCHEMA.COLUMNS
WHERE
 table_name="pageviews_2020"

---
## ジョブのメタデータ

`INFORMATION_SCHEMA.JOBS_BY_*` ビューに対してクエリを実行すると、BigQuery ジョブに関するリアルタイムのメタデータを取得できます。このビューには、現在実行中のジョブと、完了したジョブの過去 180 日間の履歴が表示されます。

- `INFORMATION_SCHEMA.JOBS_BY_USER` は、現在のプロジェクトで現在のユーザーが送信したジョブのみを返します。
- `INFORMATION_SCHEMA.JOBS_BY_PROJECT` は、現在のプロジェクトで送信されたすべてのジョブを返します。
- `INFORMATION_SCHEMA.JOBS_BY_ORGANIZATION` は、現在のプロジェクトに関連付けられている組織で送信されたすべてのジョブを返します。



`INFORMATION_SCHEMA.JOBS_BY_*` ビューのスキーマは次のとおりです。


|列名|データ型|値|
|---|---|---|
|creation_time|TIMESTAMP|（パーティショニング列）このジョブの作成時間|
|project_id|STRING|（クラスタリング列）プロジェクトの ID。|
|project_number|INTEGER|プロジェクトの番号|
|user_email|STRING||（クラスタリング列）ジョブを実行したユーザーのメールアドレスまたはサービス アカウント。|
|job_id|STRING|ジョブの ID。たとえば、`bquxjob_1234`|
|job_type|STRING|ジョブのタイプ。QUERY、LOAD、EXTRACT、COPY、または UNKNOWN のいずれかです。|
|statement_type|STRING|クエリ ステートメントのタイプ（有効な場合）。例: SELECT、INSERT、UPDATE、または DELETE。|
|start_time|TIMESTAMP|このジョブの開始時間|
|end_time|TIMESTAMP|このジョブの終了時間|
|query|STRING|SQL クエリテキスト。注: JOBS_BY_ORGANIZATION ビューには query 列がありません。|
|state|STRING|ジョブの実行状態。有効な状態は PENDING、RUNNING、および DONE です。|
|reservation_id|STRING|このジョブに割り当てられたメイン予約の名前（該当する場合）。|
|total_bytes_processed|INTEGER|ジョブによって処理された合計バイト数|
|total_slot_ms|INTEGER|ジョブの全期間のスロット（ミリ秒）。|
|error_result|RECORD|エラーの詳細（ある場合）。|
|cache_hit|BOOLEAN|クエリ結果がキャッシュされたかどうか。|
|destination_table|RECORD|結果の宛先テーブル（ある場合）。|
|referenced_tables|RECORD|ジョブによって参照されるテーブルの配列。|
|labels|RECORD|ジョブに適用されるラベルの配列。|
|timeline|RECORD|ジョブのクエリ タイムライン。クエリ実行のスナップショットが格納されます。|
job_stages|RECORD|ジョブのクエリステージ。|

BigQuery の INFORMATION_SCHEMA ジョブビューはリージョン化されています。これらのビューに対してクエリを実行するには、`region-<region_name>.INFORMATION_SCHEMA.JOBS_BY_*` という形式の関連するリージョン名を接頭辞として付ける必要があります。

利用可能なリージョンのリストは[こちら](https://cloud.google.com/bigquery/docs/locations?hl=ja)。

### サービスアカウントに必要な権限を付与する

以下のセルを実行してサービスアカウントを取得し、サービスアカウントに必要な権限を付与してください

In [ ]:
!gcloud config get-value account

手順
1. [IAM](https://console.cloud.google.com/iam-admin/iam?) にアクセス
1. 当該のサービスアカウントの右側のペンのマークをクリック
1. [Add Another Role]をクリック
1. [Select a Role] から `BigQuery Admin` を選択
1. [Save] をクリック

### 例 1: 平均スロット使用率

In [ ]:
%%bigquery
SELECT
 SUM(total_slot_ms) / (1000*60*60*24*7) AS avg_slots
FROM `region-us`.INFORMATION_SCHEMA.JOBS_BY_PROJECT
WHERE
 job_type = "QUERY"
 AND end_time BETWEEN TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 7 DAY) AND CURRENT_TIMESTAMP()

### 例 2: ジョブ履歴を読み込む

In [ ]:
%%bigquery
SELECT
 job_id, user_email
FROM
 `region-us`.INFORMATION_SCHEMA.JOBS_BY_PROJECT
WHERE
 job_type = "LOAD"

### 例 3: 最も高額なジョブを取得

In [ ]:
%%bigquery jobs
SELECT
 job_id,
 query,
 total_bytes_processed
FROM `region-us`.INFORMATION_SCHEMA.JOBS_BY_PROJECT
ORDER BY total_bytes_processed DESC
LIMIT 5

In [ ]:
jobs

In [ ]:
def print_job_processed(job):
    dollar_per_TB = 5
    print('the query below processed {} bytes and costs {} dollar\n'.format(job['total_bytes_processed'], 
                                                                     dollar_per_TB / 10**12 * job['total_bytes_processed']))
    print(job['query'])

print_job_processed(jobs.iloc[0])

---
Except as otherwise noted, the content of this page is licensed under the Creative Commons Attribution 4.0 License, and code samples are licensed under the Apache 2.0 License. For details, see the Google Developers Site Policies. Java is a registered trademark of Oracle and/or its affiliates.

